<a href="https://colab.research.google.com/github/LeoneFabio/Egocentric-Vision/blob/main/LLaVa_NeXT_Video_demo_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running LLaVa-NeXT-Video: a large multi-modal model on Google Colab

LLaVa-NeXT-Video is a new Large Vision-Language Model that enables interaction with videos and images. The model is based on a previuos series of models: [LLaVa-NeXT](https://huggingface.co/docs/transformers/main/en/model_doc/llava_next) that was trained exclusively on image-text data. The architecutre is same as in LLaVa-NeXT and is a decoder-based text model that takes concatenated vision hidden states with text hidden states.


<img src="http://drive.google.com/uc?export=view&id=1fVg-r5MU3NoHlTpD7_lYPEBWH9R8na_4">


LLaVA-NeXT surprisingly has strong performance in understanding video content with the AnyRes technique that it uses. The AnyRes technique naturally represents a high-resolution image into multiple images. This technique is naturally generalizable to represent videos because videos can be considered as a set of frames (similar to a set of images in LLaVa-NeXT). The current version of LLaVA-NeXT for videos has several improvements:

- LLaVA-Next-Video, with supervised fine-tuning (SFT) on top of LLaVA-Next on video data, achieves better video understanding capabilities and is a current SOTA among open-source models on [VideoMME bench](https://arxiv.org/pdf/2405.21075)
- LLaVA-Next-Video-DPO, which aligns the model response with AI feedback using direct preference optimization (DPO), shows further performance boost.

Transformers docs: https://huggingface.co/docs/transformers/main/en/model_doc/llava_next_video
project page: https://github.com/LLaVA-VL/LLaVA-NeXT



First we need to install the latest `transformers` from `main`, as the model has just been added. Also we'll install `bitsandbytes` to load the model in lower bits for [memory efficiency](https://huggingface.co/blog/4bit-transformers-bitsandbytes)

In [1]:
!pip install --upgrade -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 13.7 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ckvy9xhl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-ckvy9xhl
  Resolved https://github.com/huggingface/transformers.git to commit 42865860ec6dc135972d9555753cb7ee17f51fb4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10328717 sha256=117958926c577d219ab2808b7169dbd7f2af54117e8d8c8fe6ae85c96e393002
  Stored in directory: /tmp/pip-ephem-wheel-cache-xluw50oe/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:

In [2]:
# we need av to be able to read the video
!pip install -q av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 18.5 MB/s eta 0:00:00


## Load the model

Next, we load a model and corresponding processor from the hub.

We will specify a quantization config to load the model in 4 bits. Please refer to this [guide](https://huggingface.co/blog/4bit-transformers-bitsandbytes) for more details.

In [3]:
from transformers import BitsAndBytesConfig, LlavaNextVideoForConditionalGeneration, LlavaNextVideoProcessor
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

processor = LlavaNextVideoProcessor.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf")
model = LlavaNextVideoForConditionalGeneration.from_pretrained(
    "llava-hf/LLaVA-NeXT-Video-7B-hf",
    quantization_config=quantization_config,
    device_map='auto'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

## Preparing the video and image inputs

In order to read the video we'll use `av` and sample 8 frames. You can try to sample more frames if the video is long. The model was trained with 32 frames, but can ingest more as long as we're in the LLM backbone's max sequence length range.

In [4]:
import av
import numpy as np

def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.

    Args:
        container (av.container.input.InputContainer): PyAV container.
        indices (List[int]): List of frame indices to decode.

    Returns:
        np.ndarray: np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

In [5]:
import os
import json
import re

def order_videos_by_clip_uid_and_prediction(json_file_path, video_folder_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    queries_info = [
        {"clip_uid": query["clip_uid"], "best_prediction": query["best_prediction"]}
        for query in data.get("top_50_queries", [])
    ]
    video_files = [f for f in os.listdir(video_folder_path) if f.endswith('.mp4')]

    video_mapping = {}
    for video in video_files:
        video_name = os.path.splitext(video)[0]

        video_mapping[video_name] = os.path.join(video_folder_path, video)

    ordered_videos = []
    for query_info in queries_info:

        best_prediction_int = [int(val) for val in query_info["best_prediction"]]
        combined_name = f"{query_info['clip_uid']}_{best_prediction_int[0]}_{best_prediction_int[1]}"

        if combined_name in video_mapping:
            ordered_videos.append(video_mapping[combined_name])
        else:
            ordered_videos.append(None)
    return ordered_videos

json_file_path = "/content/top_50_queries.json"
video_folder_path = "/content/cut_clips"

ordered_videos = order_videos_by_clip_uid_and_prediction(json_file_path, video_folder_path)
print(ordered_videos)


['/content/cut_clips/5726971c-b3cc-43ed-8071-f6ee143e417d_0_14.mp4', '/content/cut_clips/5d531ac1-010a-4e67-ba1a-96e485b14968_0_37.mp4', '/content/cut_clips/1731de62-b1b9-4b84-bd55-412cd67e9b3c_0_18.mp4', '/content/cut_clips/9093c341-5f95-4402-a2dd-6a84aa42bf1f_0_45.mp4', '/content/cut_clips/ab094ea2-9251-4f10-945b-c2ab00c5282e_0_7.mp4', '/content/cut_clips/b8654118-84a4-4167-83c9-f268cc15f7b2_0_26.mp4', '/content/cut_clips/75d3fc52-3776-47d4-b7fd-8074d30b06d1_131_180.mp4', '/content/cut_clips/4ba774a8-cd2a-4889-9971-cc91f5c1afd4_11_15.mp4', '/content/cut_clips/1731de62-b1b9-4b84-bd55-412cd67e9b3c_41_56.mp4', '/content/cut_clips/88dcb32f-a537-47de-b3bf-f9149352bbb9_232_242.mp4', '/content/cut_clips/3672773c-6ff8-47c2-9ef9-bb00c65814ef_0_67.mp4', '/content/cut_clips/cab983c1-d36e-4afa-8116-1e2bde4a4a4c_101_116.mp4', '/content/cut_clips/2237fc47-e8c9-4751-9b02-6189913b4b4d_0_3.mp4', '/content/cut_clips/49931037-b822-4c7b-baf4-4626c1e6b6ea_41_153.mp4', '/content/cut_clips/96453857-2454-41

In [6]:
clips = []
for video in ordered_videos:
  container = av.open(video)
  # sample uniformly 8 frames from the video (we can sample more for longer videos)
  total_frames = container.streams.video[0].frames
  indices = np.arange(0, total_frames, total_frames / 20).astype(int)
  clips.append(read_video_pyav(container, indices))



In [7]:
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# np array with shape (frames, height, width, channels)
video = clips[0]

fig = plt.figure()
im = plt.imshow(video[0,:,:,:])

plt.close() # this is required to not display the generated image

def init():
    im.set_data(video[0,:,:,:])

def animate(i):
    im.set_data(video[i,:,:,:])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0],interval=100)
HTML(anim.to_html5_video())

## Prepare a prompt and generate

In the prompt, you can refer to video using the special `<video>` or `<image>` token. To indicate which text comes from a human vs. the model, one uses USER and ASSISTANT respectively (note: it's true only for this checkpoint). The format looks as follows:

`USER: <video>\n<prompt> ASSISTANT:`


In other words, you always need to end your prompt with ASSISTANT:.


Manually adding USER and ASSISTANT to your prompt can be error-prone since each checkpoint has its own prompt format expected, depending on the backbone language model. Luckily we can use `apply_chat_template` to make it easier.

Chat templates are special templates written in jinja and added to the model's config. Whenever we call `apply_chat_template`, the jinja template in filled in with your text instruction.

To use chat template simply build a list of messages, with role and content keys, and then pass it to the `apply_chat_template()` method. Once you do that, you’ll get output that’s ready to go! When using chat templates as input for model generation, it’s also a good idea to use `add_generation_prompt=True` to add a generation prompt. See [the docs](https://huggingface.co/docs/transformers/main/en/chat_templating) for more details

In [8]:
import json

json_file_path = '/content/top_50_queries.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)

conversations = []
prompts = []
for query in data.get("top_50_queries", []):
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": query["query"]},
                {"type": "video"}
            ]
        }
    ]
    conversations.append(conversation)
    # Apply the processor's chat template
    prompts.append(processor.apply_chat_template(conversation, add_generation_prompt=True))





In [9]:
print(prompts)

['USER: <video>\nWhere is the soap? ASSISTANT:', 'USER: <video>\nWhat vegetables did I chop?  ASSISTANT:', 'USER: <video>\nWhat food did I eat? ASSISTANT:', 'USER: <video>\nWhat did I put in the bowl? ASSISTANT:', 'USER: <video>\nwhere did i put my phone? ASSISTANT:', 'USER: <video>\nWhat tool did I use on the machine first? ASSISTANT:', 'USER: <video>\nWhat did I put in the dish? ASSISTANT:', 'USER: <video>\nWhere was the egg before I picked it? ASSISTANT:', 'USER: <video>\nWhat did I pour in the mug? ASSISTANT:', 'USER: <video>\nWho did I talk to at the workshop? ASSISTANT:', 'USER: <video>\nwhat did l put in the bucket? ASSISTANT:', 'USER: <video>\nIn what location did i see the car? ASSISTANT:', 'USER: <video>\nWhere is round brush before picked? ASSISTANT:', 'USER: <video>\nWhere did I wash the rice? ASSISTANT:', 'USER: <video>\nWhat did I take from the fridge? ASSISTANT:', 'USER: <video>\nwhere did  put the clothe before l hanged them? ASSISTANT:', 'USER: <video>\nHow many doughs

In [10]:

inputs = []
for prompt, clip in zip(prompts, clips):
    inputs.append(processor([prompt], videos=[clip], padding=True, return_tensors="pt").to(model.device))

In [15]:
generate_kwargs = {"max_new_tokens": 500, "do_sample": True, "top_p": 0.9}
generated_text = []
for input in inputs:
  output = model.generate(**input, **generate_kwargs)
  generated_text.append( processor.batch_decode(output, skip_special_tokens=True))

In [18]:
print(generated_text)

[["USER: \nWhere is the soap? ASSISTANT: In the image you've provided, the soap is on the top of the sink. It appears to be a bar of soap, possibly white or light-colored in appearance, placed within easy reach on the side of the sink. The sink itself is a basin-style, with a faucet on the left and the right side. There are also a number of items that appear to be dishware and cleaning products around it, indicating that this is likely a domestic or possibly a commercial kitchen environment."], ["USER: \nWhat vegetables did I chop?  ASSISTANT: It looks like you are chopping green vegetables, but it's not possible for me to identify the specific type of vegetable(s) without a clearer view. Please provide a clearer image or more details."], ["USER: \nWhat food did I eat? ASSISTANT: In the image provided, it appears that you're having a meal that includes bread and a bowl of food that might be stew or a stew-like dish, served with a spoon. However, the details of the dish are not clearly 

In [19]:
# Funzione per formattare il testo generato e includere il nome del video
def format_output(outputs, video_names):
    formatted = ""
    counter = 1  # Contatore globale per numerare le risposte

    for response_group, video_name in zip(outputs, video_names):  # Associa ogni gruppo di risposte al video
        for response in response_group:
            # Dividi la stringa in USER e ASSISTANT
            try:
                user_part, assistant_part = response.split("ASSISTANT:")
                user_question = user_part.replace("USER:", "").strip()
                assistant_answer = assistant_part.strip()

                # Aggiungi la formattazione includendo il nome del video
                formatted += (
                    f"---\n\n**{counter}. VIDEO:** {video_name}\n\n"
                    f"**USER:**\n{user_question}\n\n"
                    f"**ASSISTANT:**\n{assistant_answer}\n\n"
                )
                counter += 1
            except ValueError:
                # Se il formato non è corretto, aggiungi un avviso
                formatted += (
                    f"---\n\n**{counter}. VIDEO:** {video_name}\n\n"
                    f"**ERROR:**\nResponse format invalid: {response}\n\n"
                )
                counter += 1
    return formatted

# Esempio di utilizzo con la lista dei nomi dei video
formatted_output = format_output(generated_text, ordered_videos)

# Stampare il risultato formattato
print(formatted_output)


---

**1. VIDEO:** /content/cut_clips/5726971c-b3cc-43ed-8071-f6ee143e417d_0_14.mp4

**USER:**
Where is the soap?

**ASSISTANT:**
In the image you've provided, the soap is on the top of the sink. It appears to be a bar of soap, possibly white or light-colored in appearance, placed within easy reach on the side of the sink. The sink itself is a basin-style, with a faucet on the left and the right side. There are also a number of items that appear to be dishware and cleaning products around it, indicating that this is likely a domestic or possibly a commercial kitchen environment.

---

**2. VIDEO:** /content/cut_clips/5d531ac1-010a-4e67-ba1a-96e485b14968_0_37.mp4

**USER:**
What vegetables did I chop?

**ASSISTANT:**
It looks like you are chopping green vegetables, but it's not possible for me to identify the specific type of vegetable(s) without a clearer view. Please provide a clearer image or more details.

---

**3. VIDEO:** /content/cut_clips/1731de62-b1b9-4b84-bd55-412cd67e9b3c_0_

### Rouge and Blue Metrics

In [20]:
# Importazioni necessarie
!pip install datasets
!pip install evaluate
!pip install rouge_score
from nltk.translate.bleu_score import sentence_bleu
import evaluate

# Funzione per estrarre la risposta dell'assistente
def extract_assistant_response(generated_text):
    """
    Estrae solo la risposta dell'assistente dal testo generato.
    Assumiamo che il formato sia 'USER: ... ASSISTANT: ...'.
    """
    if "ASSISTANT:" in generated_text:
        return generated_text.split("ASSISTANT:")[1].strip()
    return generated_text.strip()

# Estrai tutte le risposte dal vettore di vettori
assistant_responses = []
for response_group in generated_text:
    for response in response_group:
        assistant_responses.append(extract_assistant_response(response))

# Ground truth
reference_text = [
    "The humor in this video stems from the unexpected and unpredictable nature of the baby's actions.",
    "In the video, there is a person wearing a white karate gi performing a technique.",
    "In this video there is a green garden in a park, with green grass and a lot of trees, the sun is shining adn sky is blue"
]

# Funzione per calcolare il punteggio BLEU
def calculate_bleu_score(candidate, references):
    references = [ref.split() for ref in references]  # Splitta in parole
    candidate = candidate.split()
    return sentence_bleu(references, candidate)

# Usa evaluate per ROUGE
rouge_metric = evaluate.load("rouge")

def calculate_rouge_score(candidate, references):
    results = rouge_metric.compute(predictions=[candidate], references=[references])
    return results

# Calcola i punteggi e formatta i risultati
formatted_results = []
for response in assistant_responses:
    bleu_score = calculate_bleu_score(response, reference_text)
    rouge_score = calculate_rouge_score(response, reference_text)

    formatted_results.append({
        "result": response,
        "bleu_score": bleu_score,
        "rouge_score": rouge_score["rouge1"]
    })

# Stampa i risultati
for i, res in enumerate(formatted_results, start=1):
    print(f"---\n**{i}. Result:**\n{res['result']}\n")
    print(f"**BLEU Score:** {res['bleu_score']:.4f}")
    print(f"**ROUGE Score:** {res['rouge_score']:.4f}\n")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
ERROR: Operation cancelled by user
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24

ModuleNotFoundError: No module named 'evaluate'